# AbstRCT Joint Task Post-Processing

## Libraries

In [244]:
import os
import pickle
import json
import pandas as pd
import ast
from sklearn.metrics import classification_report

In [281]:
nb_epochs = 5
OUTPUT_DIR = "../finetuned_models_run3/abstRCT_joint_llama-3-8b-Instruct-bnb-4bit"
DATASET_DIR = "../datasets"
split = "mix"

In [282]:
with open(os.path.join(OUTPUT_DIR, f"""abstRCT_joint_results_5_{split}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [283]:
results

{'ground_truths': ['{"component_types": ["Premise", "Premise", "Premise", "Premise", "Premise", "Claim"], "argument_relations_and_types": [[1, 6, "support"], [4, 6, "support"]]}',
  '{"component_types": ["Claim", "Premise", "Premise", "Premise", "Claim", "Claim", "Claim", "Claim"], "argument_relations_and_types": [[2, 5, "support"], [3, 5, "support"], [5, 7, "support"], [6, 8, "support"]]}',
  '{"component_types": ["Premise", "Premise", "Premise", "Premise", "Premise", "Premise", "Premise", "Claim", "Claim"], "argument_relations_and_types": [[1, 8, "support"], [2, 8, "support"], [3, 8, "support"], [5, 9, "support"], [6, 8, "support"], [7, 9, "support"]]}',
  '{"component_types": ["Claim", "Claim", "Premise", "Premise", "Premise", "Premise", "Claim", "Claim"], "argument_relations_and_types": [[3, 8, "support"], [4, 8, "support"], [5, 8, "support"], [6, 8, "support"]]}',
  '{"component_types": ["Premise", "Premise", "Premise", "Premise", "Premise", "Claim", "Claim"], "argument_relations_

In [284]:
grounds_global = results["ground_truths"]

In [285]:
preds_global = results["predictions"]

In [286]:
len(grounds_global), len(preds_global)

(100, 100)

## ACC post-processing

In [63]:
grounds = [json.loads(x)["component_types"] for x in grounds_global]

In [64]:
preds = [x["content"] for x in preds_global]

In [65]:
#preds[75] = '{"component_types": ["value", "policy", "policy", "policy", "value", "value", "policy", "policy", "policy", "policy", "value", "value", "policy", "policy", "policy", "value", "value", "policy", "policy", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy", "value", "policy", "policy", "policy"]}'

In [66]:
#preds[8] = '{"component_types": ["fact", "fact", "fact", "fact", "fact", "fact", "fact", "fact", "fact", "fact"], "argument_relations_and_types": [[1, 2, "reason"], [1, 3, "reason"], [1, 5, "reason"], [1, 6, "reason"], [1, 7, "reason"], [1, 8, "reason"], [1, 9, "reason"], [1, 10, "reason"], [2, 3, "reason"], [2, 5, "reason"], [2, 6, "reason"], [2, 7, "reason"], [2, 8, "reason"], [2, 9, "reason"], [2, 10, "reason"], [3, 5, "reason"], [3, 6, "reason"], [3, 7, "reason"], [3, 8, "reason"], [3, 9, "reason"], [3, 10, "reason"], [4, 5, "reason"], [4, 6, "reason"], [4, 7, "reason"], [4, 8, "reason"], [4, 9, "reason"], [4, 10, "reason"], [5, 6, "reason"], [5, 7, "reason"], [5, 8, "reason"], [5, 9, "reason"], [5, 10, "reason"], [6, 7, "reason"], [6, 8, "reason"], [6, 9, "reason"], [6, 10, "reason"], [7, 8, "reason"], [7, 9, "reason"], [7, 10, "reason"], [8, 9, "reason"], [8, 10, "reason"], [9, 10, "reason"], [1, 4, "reason"], [2, 4, "reason"], [3, 4, "reason"], [5, 4, "reason"], [6, 4, "reason"], [7, 4, "reason"], [8, 4, "reason"], [9, 4, "reason"], [10, 4, "reason"], [1, 3, "reason"], [2, 3, "reason"], [3, 3, "reason"], [4, 3, "reason"], [5, 3, "reason"], [6, 3, "reason"], [7, 3, "reason"], [8, 3, "reason"], [9, 3, "reason"], [10, 3, "reason"], [1, 2, "reason"], [1, 5, "reason"], [1, 6, "reason"], [1, 7, "reason"], [1, 8, "reason"], [1, 9, "reason"], [1, 10, "reason"], [2, 5, "reason"], [2, 6, "reason"], [2, 7, "reason"], [2, 8, "reason"], [2, 9, "reason"], [2, 10, "reason"], [3, 5, "reason"], [3, 6, "reason"], [3, 7, "reason"], [3, 8, "reason"], [3, 9, "reason"], [3, 10, "reason"], [4, 5, "reason"], [4, 6, "reason"], [4, 7, "reason"], [4, 8, "reason"], [4, 9, "reason"], [4, 10, "reason"], [5, 6, "reason"], [5, 7, "reason"], [5, 8, "reason"], [5, 9, "reason"], [5, 10, "reason"], [6, 7, "reason"], [6, 8, "reason"], [6, 9, "reason"], [6, 10, "reason"], [7, 8, "reason"], [7, 9, "reason"], [7, 10, "reason"], [8, 9, "reason"], [8, 10, "reason"], [9, 10, "reason"], [1, 1, "reason"], [2, 2, "reason"], [3, 3, "reason"], [4, 4, "reason"], [5, 5, "reason"], [6, 6, "reason"], [7, 7, "reason"], [8, 8, "reason"]]}'

In [67]:
preds = [json.loads(x)["component_types"] for x in preds]

In [68]:
len(grounds), len(preds)

(100, 100)

In [69]:
grounds

[['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Claim',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim',
  'Claim',
  'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim'],
 ['Claim',
  'Claim',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim', 'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  'Claim',
  'Premise',
  'Premise',
  'Claim',
  'Claim'],
 ['Claim', 'Premise', 'Premise', 'Premise', 'Claim', 'Claim'],
 ['Claim',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim',
  'Premise',
  'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim',
  'Claim',
  'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  'Prem

In [70]:
preds

[['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Claim',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim',
  'Claim',
  'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim'],
 ['Claim',
  'Claim',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Premise',
  'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim', 'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  'Claim',
  'Premise',
  'Premise',
  'Claim',
  'Claim'],
 ['Claim', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Claim',
  'Claim',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Claim',
  'Premise',
  'Premise',
  'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  'Premise',
  'Claim',
  'Premise',
  'Claim',
  'Claim'],
 ['Premise', 'Premise', 'Premise', 'Premise', 'Premise', 'Claim'],
 ['Premise',
  'Premise',
  'Premise',
  '

In [71]:
def opposite(component_type):

    if component_type == "Premise":
        return "Claim"
    elif component_type == "Claim":
        return "Premise"

In [72]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

In [73]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [74]:
ATC_preds = [item for row in preds for item in row]
ATC_grounds = [item for row in grounds for item in row]

In [75]:
# sanity check: 
len(ATC_preds) == len(ATC_grounds)

True

In [77]:
print(classification_report(ATC_grounds, ATC_preds, digits=3))

              precision    recall  f1-score   support

       Claim      0.943     0.939     0.941       212
     Premise      0.967     0.969     0.968       388

    accuracy                          0.958       600
   macro avg      0.955     0.954     0.954       600
weighted avg      0.958     0.958     0.958       600



In [78]:
with open(f"""{OUTPUT_DIR}/classification_report_acc_{split}.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(ATC_grounds, ATC_preds, output_dict=True), fh)

### ARI post-processing (TBD)

In [135]:
len(grounds_global), len(preds_global)

(100, 100)

In [136]:
def process_grounds_ari(grounds_l, nr_acs_l):

    pairs_l = []
    
    for idx, ac_count in enumerate(nr_acs_l):
        for i in range(ac_count):
            for j in range(ac_count):

                if i != j:
                    pair = [i, j]
                    if pair in grounds_l[idx]:
                        pairs_l.append([i, j, "Rel"])
                    else:
                        pairs_l.append([i, j, "N-Rel"])

    return pairs_l


def process_preds_ari(preds_l, nr_acs_l):

    pairs_l = []
    
    for idx, ac_count in enumerate(nr_acs_l):
        for i in range(ac_count):
            for j in range(ac_count):

                if i != j:
                    pair = [i, j]
                    if pair in preds_l[idx]:
                        pairs_l.append([i, j, "Rel"])
                    else:
                        pairs_l.append([i, j, "N-Rel"])

    return pairs_l

In [137]:
grounds = [json.loads(x)["argument_relations_and_types"] for x in grounds_global]

preds = [x["content"] for x in preds_global]

preds = [json.loads(x)["argument_relations_and_types"] for x in preds]

In [138]:
grounds

[[[1, 6, 'support'], [4, 6, 'support']],
 [[2, 5, 'support'], [3, 5, 'support'], [5, 7, 'support'], [6, 8, 'support']],
 [[1, 8, 'support'],
  [2, 8, 'support'],
  [3, 8, 'support'],
  [5, 9, 'support'],
  [6, 8, 'support'],
  [7, 9, 'support']],
 [[3, 8, 'support'], [4, 8, 'support'], [5, 8, 'support'], [6, 8, 'support']],
 [[1, 6, 'support'],
  [2, 6, 'support'],
  [3, 7, 'support'],
  [4, 6, 'support'],
  [5, 6, 'support']],
 [[1, 8, 'support'],
  [2, 8, 'support'],
  [3, 7, 'support'],
  [5, 4, 'attack'],
  [6, 4, 'support']],
 [[2, 6, 'support'], [3, 6, 'support']],
 [[3, 7, 'support'],
  [4, 8, 'support'],
  [5, 10, 'support'],
  [6, 10, 'support'],
  [9, 10, 'support']],
 [[1, 5, 'support'], [3, 5, 'support']],
 [[1, 5, 'support'], [2, 5, 'support'], [4, 8, 'support'], [8, 7, 'attack']],
 [[1, 6, 'support'], [2, 6, 'support'], [4, 3, 'attack']],
 [[1, 9, 'support'],
  [2, 10, 'support'],
  [3, 10, 'support'],
  [7, 11, 'support'],
  [8, 7, 'attack'],
  [9, 11, 'support'],
  [10,

In [139]:
preds

[[[1, 6, 'support'], [2, 6, 'support'], [3, 6, 'support'], [4, 6, 'support']],
 [[2, 7, 'support'],
  [3, 5, 'support'],
  [4, 8, 'support'],
  [5, 7, 'support'],
  [6, 8, 'support']],
 [[1, 8, 'support'],
  [2, 8, 'support'],
  [3, 8, 'support'],
  [5, 9, 'support'],
  [6, 9, 'support'],
  [7, 9, 'support']],
 [[3, 8, 'support'], [4, 8, 'support'], [5, 8, 'support'], [6, 8, 'support']],
 [[1, 6, 'support'],
  [2, 6, 'support'],
  [3, 7, 'support'],
  [4, 7, 'support'],
  [5, 6, 'support']],
 [[1, 4, 'support'],
  [2, 7, 'support'],
  [3, 4, 'support'],
  [5, 4, 'attack'],
  [6, 7, 'attack'],
  [7, 8, 'support']],
 [[2, 6, 'support'], [3, 6, 'support']],
 [[3, 7, 'support'], [4, 8, 'support'], [9, 10, 'support']],
 [[1, 5, 'support'], [2, 5, 'support'], [3, 5, 'support'], [4, 3, 'support']],
 [[1, 5, 'support'],
  [2, 5, 'support'],
  [3, 8, 'support'],
  [4, 8, 'support'],
  [7, 5, 'attack'],
  [8, 7, 'attack']],
 [[1, 6, 'support'],
  [2, 6, 'support'],
  [3, 6, 'support'],
  [4, 3, 

In [140]:
def get_ar_pairs(grounds):

    grounds_l = []
    
    for sublist in grounds:
        l_tmp = []
        for l in sublist:
            l_tmp.append(l[0:2])
            
        grounds_l.append(l_tmp)

    return grounds_l

In [141]:
grounds = get_ar_pairs(grounds)
preds = get_ar_pairs(preds)

In [142]:
len(grounds), len(preds)

(100, 100)

In [143]:
grounds

[[[1, 6], [4, 6]],
 [[2, 5], [3, 5], [5, 7], [6, 8]],
 [[1, 8], [2, 8], [3, 8], [5, 9], [6, 8], [7, 9]],
 [[3, 8], [4, 8], [5, 8], [6, 8]],
 [[1, 6], [2, 6], [3, 7], [4, 6], [5, 6]],
 [[1, 8], [2, 8], [3, 7], [5, 4], [6, 4]],
 [[2, 6], [3, 6]],
 [[3, 7], [4, 8], [5, 10], [6, 10], [9, 10]],
 [[1, 5], [3, 5]],
 [[1, 5], [2, 5], [4, 8], [8, 7]],
 [[1, 6], [2, 6], [4, 3]],
 [[1, 9], [2, 10], [3, 10], [7, 11], [8, 7], [9, 11], [10, 9]],
 [[2, 6], [3, 6], [4, 6], [5, 6]],
 [[1, 6], [2, 6], [3, 7]],
 [[4, 7], [7, 8]],
 [[1, 5], [2, 5], [3, 5], [4, 5]],
 [[2, 7], [3, 7], [4, 8], [5, 8], [6, 8], [7, 9], [8, 9]],
 [[3, 6], [4, 6], [5, 7]],
 [[1, 8], [2, 8], [3, 8], [4, 8], [5, 9], [6, 9], [7, 9], [8, 10], [9, 10]],
 [[1, 7], [8, 7]],
 [[1, 5], [2, 6], [3, 5], [4, 6]],
 [[1, 4], [2, 4]],
 [[2, 5], [3, 6], [4, 6]],
 [[1, 6], [2, 5], [3, 5], [4, 5]],
 [[1, 6], [3, 6], [4, 6], [5, 6], [7, 6]],
 [[1, 5], [2, 6], [3, 6], [4, 6]],
 [[1, 6], [2, 6], [3, 6], [5, 6]],
 [[1, 5], [2, 5], [3, 5], [4, 5]],
 [

In [144]:
preds

[[[1, 6], [2, 6], [3, 6], [4, 6]],
 [[2, 7], [3, 5], [4, 8], [5, 7], [6, 8]],
 [[1, 8], [2, 8], [3, 8], [5, 9], [6, 9], [7, 9]],
 [[3, 8], [4, 8], [5, 8], [6, 8]],
 [[1, 6], [2, 6], [3, 7], [4, 7], [5, 6]],
 [[1, 4], [2, 7], [3, 4], [5, 4], [6, 7], [7, 8]],
 [[2, 6], [3, 6]],
 [[3, 7], [4, 8], [9, 10]],
 [[1, 5], [2, 5], [3, 5], [4, 3]],
 [[1, 5], [2, 5], [3, 8], [4, 8], [7, 5], [8, 7]],
 [[1, 6], [2, 6], [3, 6], [4, 3], [5, 6]],
 [[1, 9], [2, 10], [3, 10], [4, 11], [5, 4], [7, 11], [8, 7], [10, 9]],
 [[1, 6], [2, 6], [3, 6], [4, 6], [5, 6]],
 [[1, 7], [2, 7], [3, 7], [5, 8], [6, 7]],
 [[4, 7], [5, 7], [6, 7], [7, 8]],
 [[1, 5], [2, 1], [3, 5], [4, 5]],
 [[1, 7], [2, 7], [3, 7], [4, 8], [5, 8], [6, 8], [7, 9], [8, 9]],
 [[2, 7], [3, 6], [4, 6], [5, 7]],
 [[1, 8], [2, 8], [3, 8], [4, 9], [5, 9], [6, 9], [7, 9], [8, 10], [9, 10]],
 [[1, 7], [2, 8], [3, 8], [4, 9], [5, 9]],
 [[1, 5], [2, 6], [3, 5], [4, 6]],
 [[1, 4], [2, 4], [3, 5]],
 [[1, 5], [2, 6], [3, 6], [4, 6]],
 [[1, 6], [2, 5], [

In [145]:
def compute_acs_nr(test_set, DATASET_DIR):

    df = pd.read_pickle(os.path.join(DATASET_DIR, f"""{test_set}_test_df.pkl"""))
    nr_acs_l = [len(element) for element in list(df.acs_list)]

    return nr_acs_l

In [146]:
nr_acs_l = compute_acs_nr(split, DATASET_DIR)

In [147]:
final_grounds = process_grounds_ari(grounds, nr_acs_l)
final_preds = process_preds_ari(preds, nr_acs_l)

In [148]:
task_grounds = [x[2] for x in final_grounds]
task_preds = [x[2] for x in final_preds]

In [149]:
# sanity check: 
len(task_grounds) == len(task_preds)

True

In [150]:
print(classification_report(task_grounds, task_preds, digits=3))

              precision    recall  f1-score   support

       N-Rel      0.987     0.971     0.979      5176
         Rel      0.629     0.791     0.701       320

    accuracy                          0.961      5496
   macro avg      0.808     0.881     0.840      5496
weighted avg      0.966     0.961     0.963      5496



In [151]:
with open(f"""{OUTPUT_DIR}/classification_report_ari_{split}.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(task_grounds, task_preds, output_dict=True), fh)

## ARC post-processing

In [287]:
grounds = [json.loads(x)["argument_relations_and_types"] for x in grounds_global]
preds = [x["content"] for x in preds_global]

In [288]:
preds = [json.loads(x)["argument_relations_and_types"] for x in preds]

In [289]:
def get_ar_pairs(grounds):

    grounds_l = []
    
    for sublist in grounds:
        l_tmp = []
        for l in sublist:
            try:
                l_tmp.append(l[2])
            except:
                l_tmp.append("Support")
            
        grounds_l.append(l_tmp)

    return grounds_l

In [290]:
grounds = get_ar_pairs(grounds)
preds = get_ar_pairs(preds)

In [291]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    if len(x) != len(y):
        print(i,x,y)

0 ['support', 'support'] ['support', 'support', 'support', 'support']
1 ['support', 'support', 'support', 'support'] ['support', 'support', 'support', 'support', 'support']
5 ['support', 'support', 'support', 'attack', 'support'] ['support', 'support', 'support', 'attack', 'attack', 'support']
7 ['support', 'support', 'support', 'support', 'support'] ['support', 'support', 'support']
8 ['support', 'support'] ['support', 'support', 'support', 'support']
9 ['support', 'support', 'support', 'attack'] ['support', 'support', 'support', 'support', 'attack', 'attack']
10 ['support', 'support', 'attack'] ['support', 'support', 'support', 'attack', 'support']
11 ['support', 'support', 'support', 'support', 'attack', 'support', 'attack'] ['support', 'support', 'support', 'support', 'attack', 'support', 'attack', 'attack']
12 ['support', 'support', 'support', 'support'] ['support', 'support', 'support', 'support', 'support']
13 ['support', 'support', 'support'] ['support', 'support', 'support', '

In [292]:
def opposite(relation_type):
    
    if relation_type == "support":
        
        return "attack"
    else:
        return "support"

In [293]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds  

In [294]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [295]:
LTC_preds = [item for row in preds for item in row]
LTC_grounds = [item for row in grounds for item in row]

In [296]:
# sanity check: 
len(LTC_preds) == len(LTC_grounds)

True

In [297]:
print(classification_report(LTC_grounds, LTC_preds, digits=3))

              precision    recall  f1-score   support

      attack      0.345     0.417     0.377        24
     support      0.952     0.936     0.944       296

    accuracy                          0.897       320
   macro avg      0.648     0.676     0.661       320
weighted avg      0.906     0.897     0.901       320



In [298]:
with open(f"""{OUTPUT_DIR}/classification_report_arc_{split}.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(LTC_grounds, LTC_preds, output_dict=True), fh)